<a href="https://colab.research.google.com/github/narhim/nn_project/blob/master/distil_pos_tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%pwd

'/content'

In [3]:
%cd /content/drive/MyDrive/NN_groupWS2021/final_project/
%pwd

/content/drive/.shortcut-targets-by-id/1JjKm5t8lyp2F4kr-Bq8HIArzsrnahAhe/NN_groupWS2021/final_project


'/content/drive/.shortcut-targets-by-id/1JjKm5t8lyp2F4kr-Bq8HIArzsrnahAhe/NN_groupWS2021/final_project'

In [4]:
import csv

In [5]:
sent_word_ids = []
sents = []
sents_pos = []
num_sents = 0
with open('sample.tsv') as file:
    #tsv_file = csv.reader(file, delimiter="\t", quotechar='"')
    word_ids = []
    words = []
    pos = []
    for line in file:
        try:
            id, word, tag = line.rstrip('\n').split('\t')
        except ValueError:
            # this will happen when you get to the line where there's only the '*' token
            num_sents += 1
            # reach the * line
            id = line.rstrip('\n').split()[0]
        if id == '*':
            assert len(word_ids) == len(words) == len(pos)
            #print(word_ids)
            #print(words)
            #print(pos)
            sent_word_ids.append(word_ids)
            sents.append(words)
            sents_pos.append(pos)
            word_ids = []
            words = []
            pos = []
        else:
            word_ids.append(int(id))
            words.append(word)
            pos.append(tag)
assert len(sent_word_ids) == len(sents) == len(sents_pos)

In [6]:
cnt = 0
for w_idx, s, t in zip(sent_word_ids, sents, sents_pos):
    print(w_idx,'\n', s, '\n', t, '\n')
    cnt += 1
    if cnt > 10:
        break

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20] 
 ['A', 'Saudi', 'woman', 'issues', 'a', 'voice', 'recording', 'in', 'which', 'she', 'mourns', 'her', 'brother', 'and', 'demands', 'the', 'punishment', 'of', 'his', 'killers', '.'] 
 ['DT', 'JJ', 'NN', 'VBZ', 'DT', 'NN', 'NN', 'IN', 'WDT', 'PRP', 'VBZ', 'PRP$', 'NN', 'CC', 'VBZ', 'DT', 'NN', 'IN', 'PRP$', 'NNS', '.'] 

[0] 
 ['HAD1999'] 
 ['NNP'] 

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56] 
 ['In', 'a', 'unique', 'way', ',', 'a', 'young', 'Saudi', 'woman', 'has', 'resorted', 'to', 'issuing', 'a', 'sound', 'tape', 'in', 'which', 'she', 'mourns', 'her', 'brother', 'murdered', 'in', 'Hafr', 'Al', 'Batin', ',', 'in', 'the', 'hope', 'of', 'rekindling', 'interest', 'in', 'his', 'case', ',', 'in', 'which', '3', 'young', 'men', 'were', 'accused',

## 80/10/10 train, val, test splits

In [7]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_tags, val_tags = train_test_split(sents, sents_pos, test_size=.2, random_state=42)
val_texts, test_texts, val_tags, test_tags = train_test_split(val_texts, val_tags, test_size=.5, random_state=42)

In [8]:
#Toy samples for faster development
#top = int(0.01*len(sents))
#sents = sents[0:top]
#sents_pos = sents_pos[0:top]
#train_texts, val_texts, train_tags, val_tags = train_test_split(sents, sents_pos, test_size=.2, random_state=42)
#val_texts, test_texts, val_tags, test_tags = train_test_split(val_texts, val_tags, test_size=.5, random_state=42)

## create encodings for our tokens and tags.

In [9]:
unique_tags = set(tag for doc in sents_pos for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}


## install transformers

In [10]:
%pip install transformers

     |████████████████████████████████| 2.0MB 23.5MB/s 
     |████████████████████████████████| 890kB 49.9MB/s 
     |████████████████████████████████| 3.2MB 51.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=b743724b7718c5d26c425e70d1ce20abb9fed0e028a9d598d7a64f9f9961a220
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


## token encodings for train and validation sets

In [12]:
#Bert
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('bert-base-cased')
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
test_encodings = tokenizer(test_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Bert and many models like it use a method called WordPiece
Tokenization, meaning that single words are split into multiple tokens such that each token is likely to be in the
vocabulary. For example, DistilBert's tokenizer would split the Twitter handle `@huggingface` into the tokens `['@',
'hugging', '##face']`. This is a problem for us because we have exactly one tag per token. If the tokenizer splits a
token into multiple sub-tokens, then we will end up with a mismatch between our tokens and our labels.

One way to handle this is to only train on the tag labels for the first subtoken of a split token. We can do this in
Transformers by setting the labels we wish to ignore to `-100`. In the example above, if the label for
`@HuggingFace` is `3` (indexing `B-corporation`), we would set the labels of `['@', 'hugging', '##face']` to
`[3, -100, -100]`.

For each sub-token returned by the tokenizer, the offset mapping gives us a tuple indicating the sub-token's
start position and end position relative to the original token it was split from. That means that if the first position
in the tuple is anything other than `0`, we will set its corresponding label to `-100`. While we're at it, we can
also set labels to `-100` if the second position of the offset mapping is `0`, since this means it must be a
special token like `[PAD]` or `[CLS]`.

In [13]:
import numpy as np

def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

In [14]:
train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)
test_labels = encode_tags(test_tags, test_encodings)

## create a dataset object:

In [15]:
import torch

class OntonetesDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [16]:
train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")
test_encodings.pop("offset_mapping")
train_dataset = OntonetesDataset(train_encodings, train_labels)
val_dataset = OntonetesDataset(val_encodings, val_labels)
test_dataset = OntonetesDataset(test_encodings, test_labels)

## check the dataset

In [17]:
len(train_dataset)

55904

In [18]:
train_dataset[0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([  101,  1109, 16028,   117,  1114,   170

## Load model for classification

In [20]:
from transformers import DistilBertForTokenClassification,BertForTokenClassification,RobertaForTokenClassification,Trainer, TrainingArguments
model = DistilBertForTokenClassification.from_pretrained('distilbert-base-cased', num_labels=len(unique_tags))

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

##Install wandb to keep track of training
User: narhim
Password: 123579trms121092
(Delete before submitting)

In [21]:
%pip install wandb
import wandb
wandb.login()
%env WANDB_PROJECT = nn_project_distil
%env WANDB_LOG_MODEL = true

     |████████████████████████████████| 2.0MB 16.7MB/s 
     |████████████████████████████████| 163kB 49.9MB/s 
     |████████████████████████████████| 102kB 13.8MB/s 
     |████████████████████████████████| 133kB 57.4MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=9f6a1d7c7d0a982815fde4472adefb08a85c6d1243d645ca4fc9117ade0f5289
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=9f4ffff39481f058f2a9758aa3b3e9fceac4a2f51877191ab8c9228720e1b3f9
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


env: WANDB_PROJECT=nn_project_distil
env: WANDB_LOG_MODEL=true


##Training and Validation

In [22]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.multioutput import MultiOutputClassifier

In [23]:

from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

#Send to CUDA
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)



#train_loader = DataLoader(train_dataset, batch_size_train, shuffle=True)
#val_loader = DataLoader(val_dataset, batch_size_val, shuffle=True)
#optim = AdamW(model.parameters(), lr=5e-5)
#
#for epoch in range(1, n_epochs + 1):
#    train(optim,train_loader,model,epoch)
#    val(val_loader,model,epoch)




DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
          

In [24]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

#model = BertForSequenceClassification.from_pretrained("bert-large-uncased")

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)
trainer.train()
trainer.evaluate()

wandb: Currently logged in as: narhim (use `wandb login --relogin` to force relogin)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:760: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Step,Training Loss
500,1.023600
1000,0.106100
1500,0.088200
2000,0.080200
2500,0.075800
3000,0.073200
3500,0.071300
4000,0.052200
4500,0.051700
5000,0.050500


Step,Training Loss
500,1.023600
1000,0.106100
1500,0.088200
2000,0.080200
2500,0.075800
3000,0.073200
3500,0.071300
4000,0.052200
4500,0.051700
5000,0.050500


wandb: WARNING Step must only increase in log calls.  Step 10482 < 10483; dropping {'eval/loss': 0.06406378746032715, 'eval/runtime': 34.9594, 'eval/samples_per_second': 199.889, 'train/epoch': 3.0}.


{'epoch': 3.0,
 'eval_loss': 0.06406378746032715,
 'eval_mem_cpu_alloc_delta': 392867,
 'eval_mem_cpu_peaked_delta': 187299,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 91972608,
 'eval_runtime': 34.9594,
 'eval_samples_per_second': 199.889}

In [32]:
#Save model
trainer.save_model()
#Make predictions
predictions, label_ids, metrics = trainer.predict(val_dataset)


In [27]:
print(metrics)

{'eval_loss': 0.06528753787279129, 'eval_runtime': 60.3078, 'eval_samples_per_second': 115.872, 'test_mem_cpu_alloc_delta': 367030842, 'test_mem_gpu_alloc_delta': 0, 'test_mem_cpu_peaked_delta': 205318, 'test_mem_gpu_peaked_delta': 719694848}


In [33]:
list_unique_tags = list(unique_tags)
list_unique_tags.sort()
predictions = np.argmax(predictions, axis=2)

In [31]:
import os
true_predictions = [[list_unique_tags[p] for (p,l) in zip(prediction, label) if l!= -100] for prediction, label in zip(predictions, label_ids)]
trainer.log_metrics("distil-test",metrics)
trainer.save_metrics("test", metrics)


output_test_predictions_file = os.path.join(training_args.output_dir,"distil_test_predictions.txt")
if trainer.is_world_process_zero():
  with open(output_test_predictions_file,"w") as writer:
    for prediction in true_predictions:
      writer.write(" ".join(prediction) + "\n")